In [14]:
import random 
import numpy as np
from typing import *

In [15]:
class GeneticAlgorithm:
    def __init__(
        self,
        pop_size: int,
        sequences: List[List[str]],
        crossover_prob: float = 0.8,
        mutation_prob: float = 0.01,
        max_iters: int = 2
    ):
        """
        Algoritmo genético para o problema do alinhamento múltiplo de sequências

        Args:
            n_sequences (int): número de sequências a serem alinhadas, dimensão do problema
            pop_size (int): tamanho da população
            sequences (List[str]): sequências a serem alinhadas
        """
        # constantes para melhor legibilidade
        self.GAP = "-"
        self.AMINO_ACIDS= set([
            'A', 'R', 'N', 'D', 'C', 'Q', 'E', 
             'G', 'H', 'I', 'L', 'K', 'M', 'F', 
             'P', 'S', 'T', 'W', 'Y', 'V'
        ])
        
        # matriz da relação de punição na troca de aminoácidos
        self.PAM250 = self.get_PAM250()
        
        # dimensões do problema
        self.n_sequences = len(sequences)
        self.pop_size = pop_size
        self.max_iters = max_iters
        
        self.mutation_prob = mutation_prob
        self.crossover_prob = crossover_prob
        
        self.population = []
        self.pop_fitness = np.zeros(pop_size) 
        
        # criando os gaps para que todos os pontos tenham o mesmo tamanho  
        # pega o tamanho da maior sequência
        largest_sequence = max(sequences, key=len)
        sequences.remove(largest_sequence)
        
        for i in range(pop_size):
            point = [largest_sequence]
            for sequence in sequences:
                while len(sequence) < len(largest_sequence):
                    idx = random.randint(0, len(sequence))
                    sequence = sequence[:idx] + [self.GAP] + sequence[idx:]
                point.append(sequence)
            
            self.population.append(point)
            self.pop_fitness[i] = self.fitness(point)
        
    
    def print_population(self):
        for point in self.population:
            print(point)
            print("\n")
            
            
    def crossover(self, P1: str, P2: str):
        operator = random.randint(1, 2) 
        
        child_1 = [None] * self.n_sequences
        child_2 = [None] * self.n_sequences
        
        idx = random.randint(0, len(P1))
        # como fazer o gap é um problema ainda
        for i in range(self.n_sequences):
            if operator == 1:
                child_1[i] = P1[i][:idx] + P2[i][idx:]
                child_2[i] = P2[i][:idx] + P1[i][idx:]
            
            elif operator == 2:
                if i < idx:
                    child_1[i] = P1[i]
                    child_2[i] = P2[i]
                else:
                    child_1[i] = P2[i]
                    child_2[i] = P1[i]
                    
        return [child_1, child_2]
    
    
    def mutation(self, P: str):
        operator = 0
        
        i = random.randint(0, self.n_sequences-1)
        sequence_size = len(P[i])
        
        if operator == 0:
            for idx in range(sequence_size-1):
                if P[i][idx] == "-":
                    for j in range(idx, sequence_size-1):
                        if P[i][j] != "-":
                            P[i][idx] = P[i][j]
                            P[i][j] = "-"
                            break
        
        return P
                    
    
    def fitness(self, point):
        sum = 0
        for i in range(len(point)-1):
            for j in range(i+1, len(point)):
                sum += self.gap_penalty(point[i], point[j])
        
        return -sum
    
    
    def gap_penalty(self, S1: str, S2: str) -> int:
        penalty = 0
        for s1, s2 in zip(S1, S2):
            if s1 == self.GAP and s2 == self.GAP:
                penalty += 1
            
            elif s1 in self.AMINO_ACIDS and s2 == self.GAP:
                penalty += 2
            
            elif s1 == self.GAP and s2 in self.AMINO_ACIDS:
                penalty += 3
            
            elif s1 in self.AMINO_ACIDS and s2 in self.AMINO_ACIDS:
                penalty += self.PAM250[s1][s2]
        
        return penalty
        
    
    def get_PAM250(self): # PAM250 scoring matrix
        sMatrixTxt = '''
        A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
        A  2 -2  0  0 -3  1 -1 -1 -1 -2 -1  0  1  0 -2  1  1  0 -6 -3
        C -2 12 -5 -5 -4 -3 -3 -2 -5 -6 -5 -4 -3 -5 -4  0 -2 -2 -8  0
        D  0 -5  4  3 -6  1  1 -2  0 -4 -3  2 -1  2 -1  0  0 -2 -7 -4
        E  0 -5  3  4 -5  0  1 -2  0 -3 -2  1 -1  2 -1  0  0 -2 -7 -4
        F -3 -4 -6 -5  9 -5 -2  1 -5  2  0 -3 -5 -5 -4 -3 -3 -1  0  7
        G  1 -3  1  0 -5  5 -2 -3 -2 -4 -3  0  0 -1 -3  1  0 -1 -7 -5
        H -1 -3  1  1 -2 -2  6 -2  0 -2 -2  2  0  3  2 -1 -1 -2 -3  0
        I -1 -2 -2 -2  1 -3 -2  5 -2  2  2 -2 -2 -2 -2 -1  0  4 -5 -1
        K -1 -5  0  0 -5 -2  0 -2  5 -3  0  1 -1  1  3  0  0 -2 -3 -4
        L -2 -6 -4 -3  2 -4 -2  2 -3  6  4 -3 -3 -2 -3 -3 -2  2 -2 -1
        M -1 -5 -3 -2  0 -3 -2  2  0  4  6 -2 -2 -1  0 -2 -1  2 -4 -2
        N  0 -4  2  1 -3  0  2 -2  1 -3 -2  2  0  1  0  1  0 -2 -4 -2
        P  1 -3 -1 -1 -5  0  0 -2 -1 -3 -2  0  6  0  0  1  0 -1 -6 -5
        Q  0 -5  2  2 -5 -1  3 -2  1 -2 -1  1  0  4  1 -1 -1 -2 -5 -4
        R -2 -4 -1 -1 -4 -3  2 -2  3 -3  0  0  0  1  6  0 -1 -2  2 -4
        S  1  0  0  0 -3  1 -1 -1  0 -3 -2  1  1 -1  0  2  1 -1 -2 -3
        T  1 -2  0  0 -3  0 -1  0  0 -2 -1  0  0 -1 -1  1  3  0 -5 -3
        V  0 -2 -2 -2 -1 -1 -2  4 -2  2  2 -2 -1 -2 -2 -1  0  4 -6 -2
        W -6 -8 -7 -7  0 -7 -3 -5 -3 -2 -4 -4 -6 -5  2 -2 -5 -6 17  0
        Y -3  0 -4 -4  7 -5  0 -1 -4 -1 -2 -2 -5 -4 -4 -3 -3 -2  0 10
    '''
        sMatrixList = sMatrixTxt.strip().split('\n')
        aaList = sMatrixList[0].split()
        sMatrix = dict()
        for aa in aaList:
            sMatrix[aa] = dict()
        for i in range(1, len(aaList) + 1):
            currRow = sMatrixList[i].split()
            for j in range(len(aaList)):
                sMatrix[currRow[0]][aaList[j]] = int(currRow[j + 1])
        return sMatrix
    
    
    def generate_L(self):
        L = []
        tau = np.zeros(self.pop_size)
            
        for i in range(self.pop_size):
            tau[i] = 1/(self.pop_fitness[i] - min(self.pop_fitness)+1)
            
        prob = tau / sum(tau)
        
        alpha = np.random.rand()
        p = prob[0]
        i = 0
        while alpha > p:
            p += prob[i]
            L.append(self.population[i])
            i+=1
        
        return L[:int(self.pop_size/2)]

    
    def form_pairs(self, L1, L2):
        pairs = []
        for l1, l2 in zip(L1, L2):
            if l1 is not None and l2 is not None:
                pairs.append([l1, l2])
        
        return pairs
            
        
    def run(self):
        for gen in range(self.max_iters):
            L1 = self.generate_L()
            L2 = self.generate_L()
            
            pairs = self.form_pairs(L1, L2)
            
            children = []
            
            for i in range(len(pairs)):
                child_1, child_2 = self.crossover(pairs[i][0], pairs[i][1])
                children.append(child_1)
                children.append(child_2)
            
            self.population = self.population + children
            self.population = [self.population[i] for i in np.argsort(self.pop_fitness)][0:self.pop_size]
            
            pop_fitness = np.zeros(self.pop_size)
            for i in range(self.pop_size):
                pop_fitness[i] = self.fitness(self.population[i])

            self.pop_fitness = pop_fitness 
            
            if np.random.rand() < self.mutation_prob:
                i = np.random.randint(0, self.pop_size-1)
                self.population[i] = self.mutation(self.population[i])
        
        return self.pop_fitness[np.argsort(self.pop_fitness)[0]], self.population[np.argsort(self.pop_fitness)[0]]
            

    def print_sequence(self, sequence):
        for line in sequence:
            print(line)

In [16]:
sequences = [
    ["G","C","P","F","S","-","S","P","N","V","E","A"],
    ["G","C","P","Y","G","C","S","P","E","A","D"]
]

ga = GeneticAlgorithm(100, sequences, max_iters=50)
ga.run()

(np.float64(-41.0),
 [['G', 'C', 'P', 'F', 'S', 'S', 'P', 'N', 'V', 'E', '-', 'A'],
  ['G', 'C', 'P', 'Y', 'G', 'C', 'S', 'P', '-', 'E', 'A', 'D']])

In [17]:
sequences = [
    ["G","C","P","F","S","-","S","P","N","V","E","A"],
    ["G","C","P","Y","G","C", "S","P","E","A","D", "-"]
]


ga.fitness(
    sequences
)

-48

In [18]:
def preprocess(sequences, out=False):
    sequences_output = []
    for sequence in sequences:
        s = []
        for letter in sequence:
            if out == False and letter == "." or letter == " ":
                continue
            s.append(letter.upper())
        sequences_output.append(s)
    
    if out:
        return sequences_output
    return list(sorted(sequences_output, key=len, reverse=True))

In [19]:
sequences = [
   "gpeeRKVYEEMAEKDKERYKREM",
    "speeKQAYIQLAKDDRIRYDNEMksweeqmae",
    "..kdKSEWEAKAATAKQNYIRALqeyerngg.",
    #"seseKAPYVAKANKLKGEYNKAIaaynkgesa"
]
sequences = preprocess(sequences)

In [20]:
def print_sequences(sequences):
    for sequence in sequences:
        for letter in sequence:
            print(letter, end="")
        print("\n")

In [21]:
print_sequences(sequences)

SPEEKQAYIQLAKDDRIRYDNEMKSWEEQMAE

KDKSEWEAKAATAKQNYIRALQEYERNGG

GPEERKVYEEMAEKDKERYKREM



In [22]:
ga.fitness(preprocess([
    "speeKQAYIQLAKDDRIRYDNEMksweeqmae",
    "--kdKSEWEAKAATAKQNYIRALqeyerngg-",
    "gpeeRKVYEEMAEKDKERYKREM---------",
], out=True))

-167

In [23]:
print_sequences(preprocess([   "gpeeRKVYEEMAEKDKERYKREM---------",
    "speeKQAYIQLAKDDRIRYDNEMksweeqmae",]))

GPEERKVYEEMAEKDKERYKREM---------

SPEEKQAYIQLAKDDRIRYDNEMKSWEEQMAE



In [24]:
ga = GeneticAlgorithm(500, sequences, max_iters=100)
fitness, sequence = ga.run()

In [25]:
fitness

np.float64(-81.0)

In [26]:
print_sequences(sequence)

SPEEKQAYIQLAKDDRIRYDNEMKSWEEQMAE

KDKSEW-E-AKAATAKQNYIRALQEYERN-GG

GPEERKV-Y--EEMAE-K-DKERYKRE--M--

